In [1]:
import pandas as pd

data_raw = pd.read_csv('../raw/지역사회건강조사 1차 전처리.csv')

In [2]:
df = data_raw[['만나이','성별','우울감 경험 여부','자치구']]

In [3]:
df_dep1=df[df['우울감 경험 여부'] != 9]
df_dep2=df_dep1[df_dep1['우울감 경험 여부'] != 7]

In [4]:
data_pbhlth_code = pd.read_excel('../raw/지역사회건강조사 2019년 원시자료 이용지침서 등/지역사회건강조사 2019 원시자료 참고사항_보건소정보, 세대유형.xlsx',header=3)
data_pbhlth_code = data_pbhlth_code[data_pbhlth_code['시도코드'] == 11][['보건소코드','보건소명']]
data_pbhlth_code["보건소명"] = data_pbhlth_code["보건소명"].str.replace('보건소', '', regex=True)
data_pbhlth_code.columns=['보건소코드','자치구']
list_region = list(data_pbhlth_code.to_dict()['자치구'].values()) + ['서울시']
list_region.reverse()

In [5]:
list_df = []
dic_all ={}
for value in list_region:
    temp = df_dep2['자치구'] == value
    list_df.append(df_dep2[temp])
list_df[0] = df_dep2

In [16]:
list_age = ['19~200','19~44','45~64','65~74','75~200']
list_sex = ['계','남','여']
idx_region = []
idx_sex = []
idx_age = []
value = []
for i, value1 in enumerate(list_region):
    for j in list_age:
        for k in list_sex:            
            temp = list_df[i][list_df[i]['만나이']>=int(value2.split('~')[0])]
            temp1 = temp[temp['만나이'] < int(value2.split('~')[1])]
            if list_age == '남':
                temp2 = temp1[temp1['성별'] == '남']
            elif list_age == '여':
                temp2 = temp1[temp1['성별'] == '여']
            else:
                temp2 = temp1
            value.append(round(temp2['우울감 경험 여부'].value_counts(normalize = True)[1] * 100, 1))
            idx_region.append(value1)
            if j == '19~200':
                idx_age.append('계')
            elif j == '75~200':
                idx_age.append('75 이상')
            else:
                idx_age.append(j)
            idx_sex.append(k)

In [17]:
df1 = pd.DataFrame({'우울감 경험률':value, '지역':idx_region,'연령별':idx_age,'성별':idx_sex})

df1 = df1.set_index(['지역','연령별','성별'])

df1

우울감 경험률
지역  연령별   성별         
서울시 계     계       7.4
          남       7.4
          여       7.4
    19~44 계       7.4
          남       7.4
...               ...
강남구 65~74 남       8.3
          여       8.3
    75 이상 계       8.3
          남       8.3
          여       8.3

[390 rows x 1 columns]